In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sys 
root_code = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, root_code)

from codebase.utils.constants import *
from codebase.utils.eval_utils import *

In [ ]:
project_path = Path('/raid/sonali/project_mvs/')
resolution = 'level_2'
metric = 'pcorr'
data_set = 'test'

In [ ]:
multi_job_id = 'mj3pqeyk_dataaug-v2-flip_split3_selected-snr_no-wt_no-checkerboard'
# single_job_ids = ['gj5tmvbv_dataaug-v2-flip_split3_TLS-set_no-wt_no-checkerboard',
#                   '2hi61lsy_dataaug-v2-flip_split3_pseudo_multiplex_ki67-correlates_no-wt_no-checkerboard',
#                  'itlm98w1_dataaug-v2-flip_split3_pseudo_multiplex_CD16-correlates_no-wt_no-checkerboard',
#                  'xyzit76o_dataaug-v2-flip_split3_pseudo_multiplex_prots-tls_no-wt_no-checkerboard',
#                  #'ksta6ini_dataaug-v2-flip_split3_CD3_single-scale_no-wt_no-checkerboard',
#                 #'boxs4un5_dataaug-v2-flip_split3_MelanA_single-scale_no-wt_no-checkerboard',
#                 'gyhp2tax_dataaug-v2-flip_split3_MelanA_singlescale_multiscaleL1_no-wt_no-checkerboard',
#                  '0jpt9ixq_dataaug-v2-flip_split3_pseudo_multiplex_no-wt_no-checkerboard',
#                   'tzav1irg_dataaug-v2-flip_split3_real-multiplex-pseudoset_no-wt_no-checkerboard',
#                  '1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard',
#                  'mjyaz45s_dataaug-v2-flip_split3_CD16-correlates_no-wt_no-checkerboard',
#                 '1id1naof_dataaug-v2-flip_split3_Ki67-correlates_no-wt_no-checkerboard']

#single_job_ids = ['1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard']

# # comparison with SOTA (note: replace MelanA with an old run as new gave empty Pcorr)
# single_job_ids = ['1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard',
#                   '6qmjfq1j_split3_otsu3_selected-snr_nocheck_multiscaleL1',
#                   '83gt7vmy_split3_otsu3_CD8a_nocheck_multiscaleL1',
#                   'dgh62fj8_split3_otsu3_CD31_nocheck_multiscaleL1',
#                   'mrv2c8gb_split3_otsu3_CD3_nocheck_multiscaleL1',
# '6qmjfq1j_split3_otsu3_selected-snr_nocheck_multiscaleL1',
# 'vphnmeya_split3_otsu3_CD16_nocheck_multiscaleL1',
# 'h7ptujju_split3_otsu3_CD20_nocheck_multiscaleL1',
# 'a1h0zpii_split3_otsu3_S100_nocheck_multiscaleL1',
#                   'gyhp2tax_dataaug-v2-flip_split3_MelanA_singlescale_multiscaleL1_no-wt_no-checkerboard',
#                   #'g7yzujop_split3_otsu3_MelanA_nocheck_multiscaleL1',
# 'j3pctguk_split3_otsu3_HLA-DR_nocheck_multiscaleL1',
# '3y5fva4l_split3_otsu3_HLA-ABC_nocheck_multiscaleL1',
# 'op0d4key_split3_otsu3_gp100_nocheck_multiscaleL1',
# 'r5tutuxj_split3_otsu3_GLUT1_nocheck_multiscaleL1']

# Comparison of hyperparameter settings for CD20
single_job_ids = ['gj5tmvbv_dataaug-v2-flip_split3_TLS-set_no-wt_no-checkerboard',
                  '1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard',
                  'tsv2bo1j_split3_otsu3_CD20_nocheck_orthogonal-init',
'8ptr128i_split3_otsu3_CD20_nocheck_xavier-init',
'qanp5d29_split3_otsu3_CD20_nocheck_kaiming-init',
'k64a5v68_split3_otsu3_CD20_nocheck_normal-init',
'fiko74v9_split3_otsu3_cd20_lrtransfixedsmall_lrdisfixedsmall',
'2uka27a1_split3_otsu3_cd20_lrtransstep_lrdisstep',
'7bcn11nu_split3_otsu3_cd20_lrtranscosine_lrdiscosine',
                 'h7ptujju_split3_otsu3_CD20_nocheck_multiscaleL1',
                 '6qmjfq1j_split3_otsu3_selected-snr_nocheck_multiscaleL1']


job_ids = single_job_ids.copy()
job_ids.append(multi_job_id)

In [ ]:
# Load metric values from all jobs
eval_df = pd.DataFrame()
for job_id in job_ids:
    epoch = get_best_epoch_w_imgs(project_path, job_id)
    eval_path = project_path.joinpath('results', job_id, data_set+'_eval', epoch, resolution, 'avgkernel_64')
    try:
        df = pd.read_csv(eval_path.joinpath(metric+'-eval.csv'), index_col=[0])
    except FileNotFoundError:
        print(job_id)
        continue
    job_args = json.load(open(eval_path.joinpath('eval_args.txt')))
    df['avg_kernel'] = job_args['avg_kernel']
    df['job_id'] = job_id
    #df['plex'] = job_id.split('_')[0] 
    df['plex'] = 'single' if job_id in single_job_ids else 'multi'
    eval_df = pd.concat([eval_df,df])

In [ ]:
# multi_job_id is always the last one, thus can use the epoch and resolution from previous cell
SAVE_PATH = project_path.joinpath('results',multi_job_id, data_set+'_pics', epoch, resolution)
if not os.path.exists(SAVE_PATH):
    SAVE_PATH.mkdir(parents=True, exist_ok=False)
save_fname = '_'.join([x.split('_')[0] for x in job_ids])
dry_run = True

In [ ]:
# Make sure that all jobs used the same averaging kernel
eval_df.avg_kernel.unique()

# Plot comparison between multi and single (if one provided) as scatterplot, colored by protein

In [ ]:
if len(single_job_ids)==1:
    plot_df = eval_df.pivot(index=['protein', 'sample_roi'], columns='plex', values='pcorr').reset_index()
    fig, ax = plt.subplots(figsize=(8,6))
    sns.scatterplot(data=plot_df, x='single', y='multi',hue='protein', palette='tab20')
    ax.plot([-0.5,1], [-0.5,1], linestyle='--', color='grey')

# SOTA comparison

In [ ]:
# sota_df = eval_df.copy()
# sota_df['plex'] = [x.replace('single', 'singleSOTA').replace('multi', 'multiMVS') for x in sota_df['plex'].to_list()]
# sota_df.loc[sota_df['job_id']=='6qmjfq1j_split3_otsu3_selected-snr_nocheck_multiscaleL1','plex'] = 'multiSOTA'
# sota_df.loc[sota_df['job_id']=='1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard','plex'] = 'singleMVS'

# plot_df = sota_df.sort_values(by=['protein','plex'])#.pivot(index=['protein', 'sample_roi'], columns='plex', values='pcorr').reset_index()
# fig, ax = plt.subplots(figsize=(8,6))
# sns.boxplot(data=plot_df, x='pcorr', y='protein',hue='plex', palette='tab10')


# CD20 comparison

In [ ]:
plot_df = eval_df.loc[eval_df['protein']=='CD20',:]
fig, ax = plt.subplots(figsize=(8,6))
sns.boxplot(data=plot_df, x='pcorr', y='job_id', dodge=False, color='lightblue')
plt.ylabel('')

# Boxplots comparing performance between jobs

In [ ]:
if len(single_job_ids)>1:
    fig, ax = plt.subplots(figsize=(6,14))
    sns.boxplot(x='pcorr', y='protein', data=eval_df, showfliers=False, ax=ax, hue='job_id', palette='tab20')
    #sns.stripplot(x='pcorr', y='protein', data=plot_df, ax=ax, hue='nuclei_slice_sim', palette='Accent', dodge=True)
    plt.ylabel('')
    plt.xlabel('Pearson correlation')
    ax.axvline(0,linestyle='--', color='grey')
    plt.legend(bbox_to_anchor=(1,1))
else:
    fig, ax = plt.subplots(figsize=(6,4))
    sns.boxplot(x='pcorr', y='protein', data=eval_df.sort_values(by=['plex','protein']), showfliers=False, ax=ax, hue='plex', palette='tab20')
    #sns.stripplot(x='pcorr', y='protein', data=plot_df, ax=ax, hue='nuclei_slice_sim', palette='Accent', dodge=True)
    plt.ylabel('')
    plt.xlabel('Pearson correlation')
    ax.axvline(0,linestyle='--', color='grey')
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()
    
    # plot only shared proteins
    shared_prots = [x for x in eval_df.loc[eval_df.plex=='single','protein'].unique() if x in eval_df.loc[eval_df.plex=='multi','protein'].unique()]
    fig, ax = plt.subplots(figsize=(6,4))
    sns.boxplot(x='pcorr', y='protein', data=eval_df.loc[eval_df.protein.isin(shared_prots),:].sort_values(by=['plex', 'protein']), showfliers=False, ax=ax, hue='plex', palette='tab20')
    #sns.stripplot(x='pcorr', y='protein', data=plot_df, ax=ax, hue='nuclei_slice_sim', palette='Accent', dodge=True)
    plt.ylabel('')
    plt.xlabel("Pearson's correlation coeff.")
    ax.axvline(0,linestyle='--', color='grey')
    plt.legend(bbox_to_anchor=(1,1))
    if not dry_run:
        plt.savefig(SAVE_PATH.joinpath(save_fname+'-pcorr_plex.png'), bbox_inches='tight', dpi=300)
        plt.savefig(SAVE_PATH.joinpath(save_fname+'-pcorr_plex.pdf'), bbox_inches='tight', dpi=300)
    plt.show()

# Export aggregated metrics to latex
Use either mean +/- std or median with IQR

In [ ]:
agg_type = 'median' #'mean'

In [ ]:
# Mean +/- std
if agg_type == 'mean':
    corr_agg = eval_df.loc[eval_df.protein.isin(shared_prots),:]
    corr_agg = corr_agg.drop(columns=['sample_roi', 'sample_id', 'roi', 'job_id']).groupby(['protein', 'plex']).pcorr.agg(['mean', 'std'])
    corr_agg = round(corr_agg,2).reset_index()
    corr_agg['mean'] = ['-' if np.isnan(x) else x for x in corr_agg['mean']]
    corr_agg['std'] = ['-' if np.isnan(x) else x for x in corr_agg['std']]
    #corr_agg['mean_std'] = [str(x)+' ('+str(y)+')' for x,y in zip(corr_agg['mean'], corr_agg['std'])]
    corr_agg['mean_std'] = [[str(x),'('+str(y)+')'] for x,y in zip(corr_agg['mean'], corr_agg['std'])]
    corr_agg = corr_agg.explode('mean_std')
    corr_agg['plex'] = [x+str(0) if i%2 == 0 else x+str(1) for i,x in enumerate(corr_agg['plex'])]
    corr_agg = corr_agg.pivot(index='plex', columns='protein', values='mean_std')
    corr_agg

In [ ]:
# Median(IQR)
if agg_type == 'median':
    def q25(x):
        return x.quantile(0.25)
    def q75(x):
        return x.quantile(0.75)
    def iqr(x):
        return x.quantile(0.75)-x.quantile(0.25)

#     corr_agg = eval_df.loc[eval_df.protein.isin(shared_prots),:]
#     corr_agg = corr_agg.drop(columns=['sample_roi', 'sample_id', 'roi', 'job_id']).groupby(['protein', 'plex']).pcorr.agg(['median', q25,q75,iqr])
#     corr_agg = round(corr_agg,2).reset_index()
#     corr_agg['median_iqr'] = [str(x)+' ('+str(y)+','+str(z)+')' for x,y,z in zip(corr_agg['median'], corr_agg['q25'], corr_agg['q75'])]
#     corr_agg['median_iqr'] = ['NaN' if np.isnan(x) else corr_agg['median_iqr'][i] for i,x in enumerate(corr_agg['median'])]
#     corr_agg = corr_agg.pivot(index='plex', columns='protein', values='median_iqr')
#     corr_agg

    corr_agg = eval_df.loc[eval_df.protein.isin(shared_prots),:]
    corr_agg = corr_agg.drop(columns=['sample_roi', 'sample_id', 'roi', 'job_id']).groupby(['protein', 'plex']).pcorr.agg(['median', q25,q75,iqr])
    corr_agg = round(corr_agg,2).reset_index()
    # print median and IQR in two consecutive lines
    corr_agg['median_iqr'] = [[str(x), '('+str(y)+','+str(z)+')'] for x,y,z in zip(corr_agg['median'], corr_agg['q25'], corr_agg['q75'])]
    corr_agg['median_iqr'] = [['-','-'] if np.isnan(x) else corr_agg['median_iqr'][i] for i,x in enumerate(corr_agg['median'])]
    corr_agg['median'] = ['-' if np.isnan(x) else x for x in corr_agg['median']]
    corr_agg = corr_agg.explode('median_iqr')
    #corr_agg['plex'] = [x+str(0) if i%2 == 0 else x+str(1) for i,x in enumerate(corr_agg['plex'])]
    corr_agg['plex'] = [x+' median' if i%2 == 0 else x+' IQR' for i,x in enumerate(corr_agg['plex'])]
    corr_agg = corr_agg.pivot(index='plex', columns='protein', values='median_iqr')
    corr_agg

In [ ]:
corr_agg_latex = corr_agg.copy()
corr_agg_latex.index = corr_agg_latex.index.get_level_values(0)
corr_agg_latex.index.name = None
corr_agg_latex.columns.name = None
display(corr_agg_latex)
for i,x in enumerate(corr_agg_latex.index):
    if x=='mean':
        corr_agg_latex.iloc[i,:] = corr_agg_latex.iloc[i,:].apply(lambda x: '\\textbf{'+str(x)+'}')
print(corr_agg_latex.to_latex(escape=False))